In [2]:
!wget https://download.openmmlab.com/mmaction/v1.0/skeleton/data/ucf101_2d.pkl -O ucf101_2d.pkl

--2025-11-16 05:27:52--  https://download.openmmlab.com/mmaction/v1.0/skeleton/data/ucf101_2d.pkl
Resolving download.openmmlab.com (download.openmmlab.com)... 47.246.23.177, 47.246.23.178, 47.246.23.179, ...
Connecting to download.openmmlab.com (download.openmmlab.com)|47.246.23.177|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1070780736 (1021M) [application/octet-stream]
Saving to: ‘ucf101_2d.pkl’

ucf101_2d.pkl       100%[===================>]   1021M  17.9MB/s    in 59s     

2025-11-16 05:28:52 (17.4 MB/s) - ‘ucf101_2d.pkl’ saved [1070780736/1070780736]



In [3]:
!pip install tqdm

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pickle
import numpy as np
from tqdm import tqdm

train_acc_final = []
val_acc_final = []
train_loss_final = []
val_loss_final = []

SELECTED_LABELS = [0, 8, 18, 25, 29, 32, 36, 59, 60, 91]

#diccionario de labels a nombres
LABEL_TO_NAME = {
    0: "ApplyLipstick",
    8: "Archery",
    18: "BodyWeightSquats",
    25: "Bowling",
    29: "CleanAndJerk",
    32: "CricketBowling",
    36: "Diving",
    59: "JumpRope",
    60: "JumpingJack",
    91: "PushUps"
}

#clases seleccionadas (en el mismo orden que SELECTED_LABELS)
SELECTED_CLASSES = [
    'ApplyLipstick',
    'Archery',
    'BodyWeightSquats',
    'Bowling',
    'CleanAndJerk',
    'CricketBowling',
    'Diving',
    'JumpRope',
    'JumpingJack',
    'PushUps'
]